In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

## Script to CSV Process

1. Load script `.txt` file.  Use [PDF Extractor](https://pdfextractor.com/) to convert any `.pdf`s to `.txt`s.
2. Remove garbage rows
3. Inspect rows removed
4. Format lines
5. Create `uppercase_unique_form.csv`
6. Manually fill out `uppercase_unique_form.csv` columns `is_character_name` and `is_dialogue` in Excel or other `.csv` editor.
7. Load `uppercase_unique.csv` and filter out rows
8. Inspect rows removed
9. Merge lines into best guesses for columns "character" and "line"
10. Manually clean the "line" column by removing text that is not dialogue.  There will be a lot, so nearly every cell will have to be cleaned!


See the script at the end of this notebook for an example of this process.

## Remove Garbage Rows

In [2]:
def remove_regex_rows(df,string):
    rows_to_remove = df["line"].str.contains(string,case=True,regex=True)
    print("Removed",rows_to_remove.sum(),"rows that match regex \"" + string + "\"")
    
    return df[~rows_to_remove], df[rows_to_remove]

def remove_whitespace_rows(df):
    whitespace_rows = df["line"].str.isspace()
    print(whitespace_rows.sum(),"whitespace rows removed")
    return df[~whitespace_rows], df[whitespace_rows]

def remove_movie_text_rows(df):
    
    regex_strings = ["^\[[^\[\]]*\]$"]
    
    removed_rows = pd.DataFrame()
    
    for regex_string in regex_strings:
        df, new_rows_to_remove = remove_regex_rows(df,regex_string)
        
        removed_rows = pd.concat([removed_rows, new_rows_to_remove])
    
    return df, removed_rows


def remove_page_number_rows(df):
    page_rows = df["line"].str.isnumeric()
    print(page_rows.sum(),"page number rows removed")
    
    return df[~page_rows], df[page_rows]

def remove_garbage_rows(df):
    df, whitespace_rows = remove_whitespace_rows(df)
    df, page_number_rows = remove_page_number_rows(df)
    df, movie_text_rows = remove_movie_text_rows(df)
    
    garbage_rows = pd.concat([whitespace_rows, page_number_rows, movie_text_rows], 
                            keys = ["whitespace","page_numbers","movie_text"])
    
    print("-----------------------------------------")
    print(garbage_rows.shape[0],"total rows removed\n")
    
    return df, garbage_rows

## Format Character Lines

In [3]:
def remove_regex(df,string):
    print("Removed",df["line"].str.count(string).sum(),"occurences of regex \"" + string + "\"")
    df["line"] = df["line"].str.replace(string,"",case=True,regex=True)

def remove_leading_trailing_whitespace(df):
    df["line"] = df["line"].str.replace("[ \t]+$","")
    df["line"] = df["line"].str.replace("^[ \t]+","")

def format_lines(df):
    remove_leading_trailing_whitespace(df)
    
    regex_strings = ["\[[^\[\]]*\]",
                     "\(O\.S\)",
                    "\(V\.O\)",
                     "\(V\.O\.\)",
                    "\(CONT.D\)",
                    "\(cont.d\)",
                    "\(O\.S\.\)",
                    "\(ON.*\)",
                    "\(ON TV\)",
                    "\(OVER.*\)",
                    "\(INTO.*\)",
                    "\(HOLO\)",
                    "\(ADR\)",
                    "\(then\)",
                    " \(RHODEY’S VOICE\)",
                    " \(PRE-LAP\)"]
    
    for regex_string in regex_strings:
        remove_regex(df, regex_string)
    
    print("")
    
    remove_leading_trailing_whitespace(df)

## Create Character / Line Columns

In [4]:
def create_character_line_columns(df):
    character = df.line.str.extract(r'(^[^:]*:)')[0]
    line = df.line.str.extract(r'(:.*$)')[0]
    
    character = character.str.replace(":$","",case=True,regex=True)
    line = line.str.replace("^[ \t]*:[ \t]*","",case=True,regex=True)
    
    character = character.str.upper()
    
    return pd.DataFrame({"character": character, "line": line})

## Ant Man

In [5]:
ant_man = pd.read_csv("./script txts/ant_man.txt",sep="\n",header=None,names=["line"])

print(ant_man.shape)

ant_man, garbage_rows = remove_garbage_rows(ant_man)
format_lines(ant_man)

ant_man = create_character_line_columns(ant_man)

ant_man.to_csv("./uncleaned/ant_man_uncleaned.csv", index=False)

ant_man = pd.read_csv("./cleaned/ant_man.csv")

ant_man.head()

#ant_man[ant_man["line"].isnull()]
#ant_man[ant_man["line"].str.contains("[^a-zA-Z\d\.,\?! '’\"-…]", regex=True)]

(1018, 1)
0 whitespace rows removed
0 page number rows removed
Removed 141 rows that match regex "^\[[^\[\]]*\]$"
-----------------------------------------
141 total rows removed

Removed 266 occurences of regex "\[[^\[\]]*\]"
Removed 0 occurences of regex "\(O\.S\)"
Removed 0 occurences of regex "\(V\.O\)"
Removed 0 occurences of regex "\(V\.O\.\)"
Removed 0 occurences of regex "\(CONT.D\)"
Removed 0 occurences of regex "\(cont.d\)"
Removed 0 occurences of regex "\(O\.S\.\)"
Removed 0 occurences of regex "\(ON.*\)"
Removed 0 occurences of regex "\(ON TV\)"
Removed 0 occurences of regex "\(OVER.*\)"
Removed 0 occurences of regex "\(INTO.*\)"
Removed 0 occurences of regex "\(HOLO\)"
Removed 0 occurences of regex "\(ADR\)"
Removed 0 occurences of regex "\(then\)"
Removed 0 occurences of regex " \(RHODEY’S VOICE\)"
Removed 0 occurences of regex " \(PRE-LAP\)"



,character,line
0,DR. HANK PYM,Stark.
1,MITCHELL CARSON,He doesn’t seem happy.
2,HOWARD STARK,"Hello, Hank. You’re supposed to be in Moscow."
3,DR. HANK PYM,I took a detour. Through your defense lab.
4,PEGGY CARTER,Tell me that isn’t what I think it is.
